In [1]:
import couler.argo as couler
from couler.argo_submitter import ArgoSubmitter


def random_code():
    import random

    res = "heads" if random.randint(0, 1) == 0 else "tails"
    print(res)


def flip_coin():
    return couler.run_script(image="python:alpine3.6", source=random_code)


def heads():
    return couler.run_container(
        image="alpine:3.6", command=["sh", "-c", 'echo "it was heads"']
    )


def tails():
    return couler.run_container(
        image="alpine:3.6", command=["sh", "-c", 'echo "it was tails"']
    )


result = flip_coin()
couler.when(couler.equal(result, "heads"), lambda: heads())
couler.when(couler.equal(result, "tails"), lambda: tails())

submitter = ArgoSubmitter()
print(couler)
couler.run(submitter=submitter)

INFO:root:Found local kubernetes config. Initialized with kube_config.
INFO:root:Checking workflow name/generatedName runpy-
INFO:root:Submitting workflow to Argo
INFO:root:Workflow runpy-6q7h5 has been submitted in "default" namespace!


<module 'couler.argo' from '/gpfs/bsc_home/xpliu/anaconda3/lib/python3.8/site-packages/couler/argo.py'>


{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'creationTimestamp': '2021-04-09T08:32:20Z',
  'generateName': 'runpy-',
  'generation': 1,
  'managedFields': [{'apiVersion': 'argoproj.io/v1alpha1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:generateName': {}},
     'f:spec': {'.': {}, 'f:entrypoint': {}, 'f:templates': {}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2021-04-09T08:32:20Z'}],
  'name': 'runpy-6q7h5',
  'namespace': 'default',
  'resourceVersion': '20199845',
  'selfLink': '/apis/argoproj.io/v1alpha1/namespaces/default/workflows/runpy-6q7h5',
  'uid': '83a90ab4-8575-4b8f-b6d9-c2efef9b83fe'},
 'spec': {'entrypoint': 'runpy',
  'templates': [{'name': 'runpy',
    'steps': [[{'name': 'flip-coin-28', 'template': 'flip-coin'}],
     [{'name': 'heads-29',
       'template': 'heads',
       'when': '{{steps.flip-coin-28.outputs.result}} == heads'},
      {'name': 'tails-30',
       'template': 'ta

In [2]:
import yaml
import pyaml

with open('./store_file.yaml', 'w') as file:
    documents = yaml.dump(couler.states.workflow, file)


In [4]:
couler.delete("runpy-5ghvx")

INFO:root:Found local kubernetes config. Initialized with kube_config.


{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'runpy-5ghvx',
  'group': 'argoproj.io',
  'kind': 'workflows',
  'uid': 'eb431d22-3f67-44cc-9200-cc116b082515'}}

In [2]:
couler.workflow_yaml()

OrderedDict([('apiVersion', 'argoproj.io/v1alpha1'),
             ('kind', 'Workflow'),
             ('metadata', {'generateName': 'runpy-'}),
             ('spec',
              {'entrypoint': 'runpy',
               'templates': [{'name': 'runpy', 'steps': {}}]})])

In [12]:
couler.init_yaml_dump()

In [13]:
couler._dump_yaml()

In [3]:
import couler.argo as couler
from couler.argo_submitter import ArgoSubmitter


def job_a(message):
    couler.run_container(
        image="docker/whalesay:latest",
        command=["cowsay"],
        args=[message],
        step_name="A",
    )


def job_b(message):
    couler.run_container(
        image="docker/whalesay:latest",
        command=["cowsay"],
        args=[message],
        step_name="B",
    )


def job_c(message):
    couler.run_container(
        image="docker/whalesay:latest",
        command=["cowsay"],
        args=[message],
        step_name="C",
    )


def job_d(message):
    couler.run_container(
        image="docker/whalesay:latest",
        command=["cowsay"],
        args=[message],
        step_name="D",
    )

#     A
#    / \
#   B   C
#  /
# D
def linear():
    couler.set_dependencies(lambda: job_a(message="A"), dependencies=None)
    couler.set_dependencies(lambda: job_b(message="B"), dependencies=["A"])
    couler.set_dependencies(lambda: job_c(message="C"), dependencies=["A"])
    couler.set_dependencies(lambda: job_d(message="D"), dependencies=["B"])


#   A
#  / \
# B   C
#  \ /
#   D
def diamond():
    couler.dag(
        [
            [lambda: job_a(message="A")],
            [lambda: job_a(message="A"), lambda: job_b(message="B")],  # A -> B
            [lambda: job_a(message="A"), lambda: job_c(message="C")],  # A -> C
            [lambda: job_b(message="B"), lambda: job_d(message="D")],  # B -> D
            [lambda: job_c(message="C"), lambda: job_d(message="D")],  # C -> D
        ]
    )


linear()
submitter = ArgoSubmitter()
couler.run(submitter=submitter)

INFO:root:Found local kubernetes config. Initialized with kube_config.
INFO:root:Checking workflow name/generatedName runpy-
INFO:root:Submitting workflow to Argo
INFO:root:Workflow runpy-5stcs has been submitted in "default" namespace!


{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'creationTimestamp': '2021-04-08T09:32:52Z',
  'generateName': 'runpy-',
  'generation': 1,
  'managedFields': [{'apiVersion': 'argoproj.io/v1alpha1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:generateName': {}},
     'f:spec': {'.': {}, 'f:entrypoint': {}, 'f:templates': {}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2021-04-08T09:32:52Z'}],
  'name': 'runpy-5stcs',
  'namespace': 'default',
  'resourceVersion': '19690930',
  'selfLink': '/apis/argoproj.io/v1alpha1/namespaces/default/workflows/runpy-5stcs',
  'uid': '64c09bb3-5ce7-4018-9bca-2cd56fcc94ec'},
 'spec': {'entrypoint': 'runpy',
  'templates': [{'dag': {'tasks': [{'arguments': {'parameters': [{'name': 'para-A-0',
          'value': 'A'}]},
       'name': 'A',
       'template': 'A'},
      {'arguments': {'parameters': [{'name': 'para-B-0', 'value': 'B'}]},
       'dependencies': ['A'],
       'n

In [3]:
import couler.argo as couler
from couler.argo_submitter import ArgoSubmitter


def job(message, stepname):
    task = couler.run_container(
        image="docker/whalesay:latest",
        command=["cowsay"],
        args=[message],
        step_name=stepname,
    )
    return task

task = job(message="A", stepname="A")

#     A
#    / \
#   B   C
#  /
# D
def linear():
    couler.set_dependencies(lambda: task, dependencies=None)
    couler.set_dependencies(lambda: job(message="B", stepname="B"), dependencies=["A"])


    
linear()


submitter = ArgoSubmitter()
couler.run(submitter=submitter)

TypeError: 'NoneType' object is not iterable

In [4]:
import os

import couler.argo as couler
from couler.argo_submitter import ArgoSubmitter
from couler.core.templates.volume_claim import VolumeClaimTemplate
from couler.core.templates.volume import VolumeMount
from couler.core.syntax import create_workflow_volume


def touch_op(data_mount):
    command = ["touch", os.path.join(data_mount.mount_path, 'hello_world.txt')]
    return couler.run_container(
        image="alpine:3.12.0", command=command, volume_mounts=[data_mount]
    )


def ls_op(data_mount):
    command = ["ls", data_mount.mount_path]
    return couler.run_container(
        image="alpine:3.12.0", command=command, volume_mounts=[data_mount]
    )


volume_name = "pvc-test"
volume_path = "/data/"
pvc = VolumeClaimTemplate(volume_name)
mount = VolumeMount(volume_name, volume_path)
create_workflow_volume(pvc)
touch_op(mount)
ls_op(mount)

submitter = ArgoSubmitter(namespace="argo")
couler.run(submitter=submitter)

INFO:root:Found local kubernetes config. Initialized with kube_config.
INFO:root:Checking workflow name/generatedName runpy-
INFO:root:Submitting workflow to Argo
INFO:root:Workflow runpy-qcmvh has been submitted in "argo" namespace!


{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'creationTimestamp': '2021-04-08T15:05:36Z',
  'generateName': 'runpy-',
  'generation': 1,
  'managedFields': [{'apiVersion': 'argoproj.io/v1alpha1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:generateName': {}},
     'f:spec': {'.': {},
      'f:entrypoint': {},
      'f:templates': {},
      'f:volumeClaimTemplates': {}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2021-04-08T15:05:36Z'}],
  'name': 'runpy-qcmvh',
  'namespace': 'argo',
  'resourceVersion': '19813708',
  'selfLink': '/apis/argoproj.io/v1alpha1/namespaces/argo/workflows/runpy-qcmvh',
  'uid': '5071a5ea-5b82-4c08-b03e-9b36a8d61abf'},
 'spec': {'entrypoint': 'runpy',
  'templates': [{'dag': {'tasks': [{'dependencies': ['A'],
       'name': 'touch-op-None',
       'template': 'touch-op'},
      {'dependencies': ['touch-op-None'],
       'name': 'ls-op-None',
       'template': 'ls-op'}]},
    

In [2]:
import os

import couler.argo as couler
from couler.argo_submitter import ArgoSubmitter
from couler.core.templates.volume import VolumeMount, Volume

couler.add_volume(Volume("apppath", "mnist"))

mount = VolumeMount("apppath", "/data/")
command = ["ls", mount.mount_path]

couler.run_container(
        image="alpine:3.12.0", command=command, volume_mounts=[mount]
    )

submitter = ArgoSubmitter(namespace="testagent")
couler.run(submitter=submitter)

INFO:root:Found local kubernetes config. Initialized with kube_config.
INFO:root:Checking workflow name/generatedName runpy-
INFO:root:Submitting workflow to Argo
INFO:root:Workflow runpy-x2lkw has been submitted in "testagent" namespace!


OrderedDict([('apiVersion', 'argoproj.io/v1alpha1'), ('kind', 'Workflow'), ('metadata', {'generateName': 'runpy-'}), ('spec', {'entrypoint': 'runpy', 'volumes': [OrderedDict([('name', 'apppath'), ('persistentVolumeClaim', {'claimName': 'mnist'})])], 'templates': [{'name': 'runpy', 'steps': [[OrderedDict([('name', 'module-3418'), ('template', 'module')])]]}, OrderedDict([('name', 'module'), ('container', OrderedDict([('image', 'alpine:3.12.0'), ('command', ['ls', '/data/']), ('volumeMounts', [OrderedDict([('name', 'apppath'), ('mountPath', '/data/')])])])), ('volumes', [])])]})])


{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'creationTimestamp': '2021-04-09T16:35:39Z',
  'generateName': 'runpy-',
  'generation': 1,
  'managedFields': [{'apiVersion': 'argoproj.io/v1alpha1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:generateName': {}},
     'f:spec': {'.': {},
      'f:entrypoint': {},
      'f:templates': {},
      'f:volumes': {}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2021-04-09T16:35:39Z'}],
  'name': 'runpy-x2lkw',
  'namespace': 'testagent',
  'resourceVersion': '20379154',
  'selfLink': '/apis/argoproj.io/v1alpha1/namespaces/testagent/workflows/runpy-x2lkw',
  'uid': '9dea87f5-df41-4a8a-89e3-27cd4d396343'},
 'spec': {'entrypoint': 'runpy',
  'templates': [{'name': 'runpy',
    'steps': [[{'name': 'module-3418', 'template': 'module'}]]},
   {'container': {'command': ['ls', '/data/'],
     'image': 'alpine:3.12.0',
     'volumeMounts': [{'mountPath': '/data/', 'name': '